# Introduction
Hey,community! It's been a lot since I've written my last kernel and I've missed to write kernels in Kaggle a lot.  Before starting, I want to share my excitement, in all of those science fiction books and movies, we've been seeing the effects of artifical intelligince to our health. Some robots and AIs were detecting problems and creating solutions. And now we're living that age, we can determine coronavirus using chest x-rays and artifical intelligince.

In this kernel I'm going to show you how to create a image classifier from scratch using Pytorch and how to train it as well.

# Table of Content
1. Preparing Dataset
1. Creating Model Architecture
1. Training Model
1. Checking Results
1. Conclusion

In [ ]:
# Basic data manipulations
import pandas as pd
import numpy as np

# Handling images
from PIL import Image
import matplotlib.pyplot as plt

# Handling paths
from glob import glob

import time

# Pytorch essentials
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Pytorch essentials for datasets.
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Pytorch way of data augmentation.
import torchvision
import torchvision.transforms as trfs

# Preparing Dataset
In this section we're gonna prepare our dataset, we'll create a class inherited by Dataset class of Pytorch. Also we'll create a sampler and combine them in a data loader.

But first, let's check our metadata.

In [ ]:
meta_data = pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')
meta_data.drop("Unnamed: 0",axis=1,inplace=True)
meta_data.head()

* Now let's create our data class.

In [ ]:
class CoronaHackDataset(Dataset):
    
    def __init__(self,image_paths,image_labels,image_size,transforms_):
        self.image_paths = image_paths
        self.image_labels = image_labels
        self.image_size = image_size
        
        # We'll use transforms for data augmentation and converting PIL images to torch tensors.
        self.transforms_ = transforms_
        
    def __len__(self):
        return len(self.meta_data)
    
    def __getitem__(self,index):
        img = Image.open(self.image_paths[index]).resize(self.image_size).convert("LA")
        transformed_img = self.transforms_(img)
        return transformed_img,self.image_labels[index]
        

* As you can see we need paths to make this class usefull. Now we'll concatenate paths and get labels.

In [ ]:
TRAIN_IMAGE_FOLDER_PATH = "../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/"
TEST_IMAGE_FOLDER_PATH = "../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/"

train_image_paths = [TRAIN_IMAGE_FOLDER_PATH+meta_data.iloc[i]["X_ray_image_name"] for i in range(len(meta_data)) if meta_data.iloc[i]["Dataset_type"] == "TRAIN"]
test_image_paths = [TEST_IMAGE_FOLDER_PATH+meta_data.iloc[i]["X_ray_image_name"] for i in range(len(meta_data)) if meta_data.iloc[i]["Dataset_type"] == "TEST"]

* And now let's check an image per set by showing them.

In [ ]:
plt.imshow(Image.open(train_image_paths[0]))
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(Image.open(test_image_paths[0]))
plt.axis("off")
plt.show()

* Alright, now let's get our labels, but before getting them we'll encode them.

In [ ]:
np.unique(meta_data["Label"])

In [ ]:
# I know, those comprehensions are kind of annoying if you're a new in Python, it simply checks data whether it's in train set
# If it's in train set and label is Normal it appends a 0, else it appends a 1
train_labels = [0 if meta_data.iloc[i]["Label"] == "Normal" else 0 for i in range(len(meta_data)) if meta_data.iloc[i]["Dataset_type"] == "TRAIN"]

test_labels = [0 if meta_data.iloc[i]["Label"] == "Normal" else 0 for i in range(len(meta_data)) if meta_data.iloc[i]["Dataset_type"] == "TEST"]

* Even though there are specified test and train set, now I'll concatenate them and recreate those sets.

In [ ]:
image_paths = train_image_paths + test_image_paths
image_labels = train_labels + test_labels

print("Number of image paths",len(image_paths))
print("Number of image labels",len(image_labels))


* It seems okay, now we'll split images indices as train and test set.

Such as index "0" might be the part of train set, and "32" might be the part of test set.

In [ ]:
from sklearn.model_selection import train_test_split
train_indices,test_indices = train_test_split(list(range(len(image_paths))),test_size=0.25,random_state=42)

print("Number of train samples",len(train_indices))
print("Number of test samples",len(test_indices))

In [ ]:
train_indices[:5]

* Images in those indices will be in train set.
* Our train and test indices, image paths and labels are determined. We're ready to create our sampler.

**Briefly, data sampler will sample random data from the indices given.**

In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

* And now we're gonna combine our sampler and dataset class in a data loader.

I want to explain it more detailed. Data loader will iterate the data we use in training and testing.

First, it'll get random indices from **data sampler**, then it'll send those indices to the **dataset class** and get images,labels. Then it'll return them and we use them :)

In [ ]:
# You can find all transforms in:
# https://pytorch.org/vision/stable/transforms.html

transforms_ = trfs.Compose([
    # This will make PIL Images Pytorch tensors.
    trfs.ToTensor(),
    trfs.RandomHorizontalFlip(),
    trfs.RandomVerticalFlip()
])


In [ ]:
dataset = CoronaHackDataset(image_paths,image_labels,(224,224),transforms_)

In [ ]:
# In each iteration there will be 128 images.
BATCH_SIZE = 128

train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, 
                                           sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE,
                                                sampler=test_sampler)

* Ahoyy, our data is ready to use, now we can start to build our deep learning model using nn module of Pytorch.

# Creating Model Architecture
In this section we're gonna create our model architecture. In Pytorch, things are generally based on **Object Oriented Programming**, so we generally create classes and use objects. 

Now we'll create our model class which is inherited by nn.Module class.

In [ ]:
class NetworkCNN(nn.Module):
    
    def __init__(self):
        super(NetworkCNN,self).__init__()
        """
        We've defined our layers as attributes here.
        """
        self.conv1 = nn.Conv2d(2,32,kernel_size=3,stride=2,padding=1)
        self.conv2 = nn.Conv2d(32,128,kernel_size=3,stride=2,padding=1)
        self.batch_norm1= nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(128,256,kernel_size=3,stride=2,padding=1)
        self.conv4 = nn.Conv2d(256,512,kernel_size=3,stride=2,padding=1)
        self.batch_norm2 = nn.BatchNorm2d(512)
        
        self.max_pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(512 * 3 * 3,2)
        
    def forward(self,x):
        """
        When we send values to the model, this function will work.
        """
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.batch_norm1(x)
        x = self.max_pool(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.batch_norm2(x)
        x = self.max_pool(x)
        
        # Flattening | layer.Flatten() equal of Keras.
        x = x.view(-1, 512 * 3 * 3)
        x = self.fc1(x)
        
        return x

* And yeah, our model class is ready, but we still need to define some objects 

In [ ]:
# We'll use GPU on training so
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device

In [ ]:
model = NetworkCNN().to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer which will use gradients to train model.
optimizer = optim.RMSprop(model.parameters(),lr=1e-3)

* Our model is ready to use as well, now we can start to train our model :)

# Training Model
In this section we're gonna train our model.

In [ ]:
#I've added this to stop the other part of kernel :D
input()

In [ ]:
EPOCH_NUMBER = 5
TRAIN_LOSS = []
TRAIN_ACCURACY = []

for epoch in range(1,EPOCH_NUMBER + 1):
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    for (data_,target_) in train_loader:
        
        target_ = target_.to(device)
        data_ = data_.to(device)
        
        # First we'll clean the cache of optimizer
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(data_)
        
        # Computing loss 
        loss = criterion(outputs,target_)
        
        # Backward propagation
        loss.backward()
        
        # Optimizing model
        optimizer.step()
        
        # Computing statistics.
        epoch_loss = epoch_loss + loss.item()
        _,pred = torch.max(outputs,dim=1)
        correct = correct + torch.sum(pred == target_).item()
        total += target_.size(0)
        
    # Appending stats to the lists.
    TRAIN_LOSS.append(epoch_loss)
    TRAIN_ACCURACY.append(100 * correct / total)
    print(f"Epoch {epoch}: Accuracy: {100 * correct/total}, Loss: {epoch_loss}")
        

* For now I dont have enough time to train this cute model, so I'll pass this section, but if you have time you can fork and train :)

# Final Test
In this section we're gonna test our model using our test loader.

In [ ]:
total_val_loss = 0
correct = 0
total = len(test_sampler)

# This means make all require_grad flags False
with torch.no_grad():
    
    # This will disable backward propagation
    model.eval()
    for data_,target_ in test_loader:
        
        data_ = data_.to(device)
        target_ = target_.to(device)
        
        # Forward propagation
        outputs = model(data_)
        
        # Computing loss
        loss = criterion(outputs,target_).item()
        total_val_loss += loss
        
        # Computing accuracy
        _,preds = torch.max(outputs,dim=1)
        true = torch.sum(preds == target_).item()
        total_true += true

validation_accuracy = round(100 * total_true / total,2)
print(f"Validation accuracy: {validation_accuracy}%")
print(f"Validation loss: {round(total_val_loss,2)}%")

# Conclusion
It has been a lot since I did not write any kernel but now I've written this kernel and feel freshed. If you liked this kernel and upvoted I've been glad. 

Have a good day!